# Animation example

In [ ]:
#@title Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    outputs_path = "/content/gdrive/MyDrive/AI/StabilityAnimations"
    !mkdir -p $outputs_path
except:
    outputs_path = "."
print(f"Animations will be saved to {outputs_path}")

In [2]:
%%capture
#@title Connect to the Stability API

import datetime
import json
import os
import panel as pn
import param
import shutil
import sys

from base64 import b64encode
from IPython import display
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from types import SimpleNamespace


# install Stability SDK for Python
path = Path('stability-sdk')
if path.exists():
    shutil.rmtree(path)
    !pip uninstall -y stability-sdk
!git clone -b animation-staging --recurse-submodules https://github.com/Stability-AI/stability-sdk
Path("./stability-sdk/src/stability_sdk/interfaces/__init__.py").touch()
!pip install ./stability-sdk[anim]


from stability_sdk.api import Context
from stability_sdk.animation import AnimationArgs, Animator
from stability_sdk.utils import create_video_from_frames


# Connect to Stability API
GRPC_HOST = "" #@param {type:"string"}
API_KEY = "" #@param {type:"string"}

api_context = Context(GRPC_HOST, API_KEY)

In [ ]:
# @title Settings

# @markdown Run this cell to reveal the settings UI. After entering values, move on to the next step.

# @markdown To reset values to default, simply re-run this cell.

# @markdown NB: Settings are grouped across several tabs.

show_documentation = True # @param {type:'boolean'}

# #@markdown ####**Resume:**
resume_timestring = "" #@param {type:"string"}

#@markdown ####**Override Settings:**
override_settings_path = "" #@param {type:"string"}

###################

from stability_sdk.animation import (
    AnimationArgs,
    Animator,
    AnimationSettings,
    BasicSettings,
    CoherenceSettings,
    ColorSettings,
    DepthSettings,
    InpaintingSettings,
    Rendering3dSettings,
    CameraSettings,
    VideoInputSettings,
    VideoOutputSettings,
)

args_generation = BasicSettings()
args_animation = AnimationSettings()
args_camera = CameraSettings()
args_coherence = CoherenceSettings()
args_color = ColorSettings()
args_depth = DepthSettings()
args_render_3d = Rendering3dSettings()
args_inpaint = InpaintingSettings()
args_vid_in = VideoInputSettings()
args_vid_out = VideoOutputSettings()
arg_objs = (
    args_generation,
    args_animation,
    args_camera,
    args_coherence,
    args_color,
    args_depth,
    args_render_3d,
    args_inpaint,
    args_vid_in,
    args_vid_out,
)

def _show_docs(component):
    cols = []
    for k, v in component.param.objects().items():
        if k == 'name':
            continue
        col = pn.Column(v, v.doc)
        cols.append(col)
    return pn.Column(*cols)

def build(component):
    if show_documentation:
        component = _show_docs(component)
    return pn.Row(component, width=1000)

pn.extension()

#pn.Tabs(*[(a.name[:-5], pn.Row(a.param,width=1000)) for a in arg_objs])
pn.Tabs(*[
    (a.name[:-5], build(a)) for a in arg_objs
])

### Prompts

In [4]:
animation_prompts = {
    0: "a painting of a delicious cheeseburger by Tyler Edlin",
    24: "a painting of the the answer to life the universe and everything by Tyler Edlin",
}

negative_prompt = ""
negative_prompt_weight = -1.0


In [ ]:
#@title Render the animation

args_d = {}
[args_d.update(a.param.values()) for a in arg_objs]
args=AnimationArgs(**args_d)


# load override settings if provided
if override_settings_path:
    if not os.path.exists(override_settings_path):
        raise ValueError(f"Override settings file not found: {override_settings_path}")
    with open(override_settings_path, 'r') as f:
        overrides = json.load(f)
        args = vars(args)
        for k in args.keys():
            if k in overrides:
                args[k] = overrides[k]
        args = SimpleNamespace(**args)
        animation_prompts = overrides.get('animation_prompts', animation_prompts)
        animation_prompts = {int(k): v for k, v in animation_prompts.items()}
        negative_prompt = overrides.get('negative_prompt', negative_prompt)
        negative_prompt_weight = overrides.get('negative_prompt_weight', negative_prompt_weight)

# create folder for frames output
if resume_timestring:
    out_dir = os.path.join(outputs_path, resume_timestring)
    if not os.path.exists(out_dir):
        raise Exception("Can't resume {resume_timestring} because path {out_dir} doesn't exist. Please make sure the timestring is correct.")
    timestring = resume_timestring
else:
    timestring = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    out_dir = os.path.join(outputs_path, timestring)
    os.makedirs(out_dir, exist_ok=True)
print(f"Saving animation frames to {out_dir}...")

animator = Animator(
    api_context=api_context,
    animation_prompts=animation_prompts,
    args=args,
    out_dir=out_dir,    
    negative_prompt=negative_prompt,
    negative_prompt_weight=negative_prompt_weight,
    resume=len(resume_timestring) != 0,
)
animator.save_settings(f"{timestring}_settings.txt")

for frame in tqdm(animator.render(), initial=animator.start_frame_idx, total=args.max_frames):
    display.clear_output(wait=True)
    display.display(frame)

In [ ]:
#@title Create video from frames
skip_video_for_run_all = False #@param {type: 'boolean'}
fps = 12 #@param {type:"number"}

if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    print(f"Compiling animation frames to {mp4_path}...")
    mp4_path = os.path.join(out_dir, f"{timestring}.mp4")
    create_video_from_frames(out_dir, mp4_path, fps)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )